<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/implicit_deriv_radau_optimal_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax.numpy as jnp
import jax
jax.config.update('jax_enable_x64', True)
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp
from numpy.polynomial.legendre import leggauss
from scipy.optimize import root, minimize
from functools import partial

In [157]:
a=0.01
A = 0.2
qin_initial_ss = 0.1
sq2g = jnp.sqrt(2*9.81)
h10 = h20=qin_initial_ss**2/(a**2)/(2*9.81)
eint0 = 0.
y0=jnp.array([h10, h20, eint0])
hsp = 2.*h20
tend=150.
Nu=25
ut = jnp.linspace(0.,tend,Nu)
u0 = jnp.full_like(ut,0.05)
bounds = [(0.,1.)]*Nu
Ny = y0.size


@jax.jit
def rhs(t, v, u):
    h1,h2,eint = v
    qin=qin_initial_ss
    qcontrol= jnp.interp(t,ut, u)
    q12 = a*sq2g*(h1**0.5)
    q2 = a*sq2g*(h2**0.5)
    return jnp.array( [(qin + qcontrol - q12)/A, (q12 - q2)/A, (h2-hsp)**2])

rhs_jac=jax.jit(jax.jacobian(rhs,1))

rhs_vec=jnp.vectorize(rhs,signature=f'(),(2),({Nu})->(2)')

In [158]:
res=solve_ivp(rhs, (0,tend), y0, method='Radau', dense_output=True, jac=jax.jacobian(rhs,1), args=(u0,))


In [159]:
x = (np.r_[-1.,leggauss(3)[0],1])/2 + 0.5
p = jnp.tile(x, (3,1))
p = jnp.cumprod(p,axis=0)
dp = p[:-1,:]*jnp.arange(2,4)[:,None]

def get_collocation_eqs(res):
    ts=jnp.array(res.t)
    hs=ts[1:]-ts[:-1]
    y0s=jnp.array(res.y)
    Qs=jnp.stack([s.Q for s in res.sol.interpolants])
    Nb = Qs.shape[0]

    flatres0 = jnp.concatenate([Qs.ravel(),res.y[:,1:].ravel()])

    def collocation_eqs(v,u):
        qflat,yb0=v[:Nb*Ny*3],v[Nb*Ny*3:]
        Qs=jnp.reshape(qflat, (Nb, Ny, 3))
        yb0=jnp.reshape(yb0,(Ny,Nb)).T
        yb0=jnp.concatenate([y0.reshape(1,-1),yb0],axis=0)
        qp=jnp.einsum('byi, ix -> byx', Qs, p)
        yb=qp[:,:,1:]+yb0[:-1,:,None]
        tb=(hs[:,None]*x[None,1:-1]+ts[:-1,None])
        ybp=(jnp.einsum('byi, ix -> byx', Qs[:,:,1:] , dp) + Qs[:,:,0][:,:,None])/hs[:,None,None]
        ybp=ybp[:,:,1:-1]
        end_equality=jnp.ravel(yb[:,:,-1]-yb0[1:,:])
        collocation=jnp.ravel(jnp.swapaxes(ybp,1,2)-rhs_vec(tb,jnp.swapaxes(yb[:,:,:-1],1,2),u))
        return jnp.concatenate([end_equality, collocation])

    def _interpolate(t,v):
        qflat,yb0=v[:Nb*Ny*3],v[Nb*Ny*3:]
        Qs=jnp.reshape(qflat, (Nb, Ny, 3))
        yb0=jnp.reshape(yb0,(Ny,Nb))
        yb0=jnp.concatenate([y0.reshape(Ny,1),yb0],axis=1)
        t=jnp.atleast_1d(t)
        i=jnp.searchsorted(ts,t, side='right')-1
        x = (t-ts[i])/hs[i]
        p = jnp.cumprod(jnp.tile(x, (3,1)),axis=0)
        y= jnp.where(t<ts[-1],jnp.einsum('tyi, it -> yt', jnp.take(Qs, i, 0), p)+jnp.take(yb0,i,1),jnp.take(yb0,i,1))
        return jnp.squeeze(y)

    jacv=jax.jacobian(collocation_eqs,0)
    jacu=jax.jacobian(collocation_eqs,1)

    return collocation_eqs, _interpolate, flatres0, jacv, jacu


In [160]:
def obj(u):

    global flatres
    res=solve_ivp(rhs, (0,tend), y0, method='Radau', dense_output=True, jac=rhs_jac, args=(u,))
    collocation_eqs, _interpolate, flatres, jacv, jacu = get_collocation_eqs(res)
    Nb=res.t.size-1

    def integrated_error(v):
        return _interpolate(tend,v)[-1]
    derrdq = jax.jacobian(integrated_error)

    solve=root(collocation_eqs, flatres, jac=jacv,args=(u,))
    flatres=solve.x
    Hu=jacu(flatres,u)
    Hq=jacv(flatres,u)
    dqdu=-(jnp.linalg.inv(Hq) @ Hu)
    err=integrated_error(flatres)
    grad=derrdq(flatres)@dqdu
    print(err)
    return err,grad

In [161]:
sol=minimize(obj,u0, jac=True, bounds=bounds)

710.5794929575727
391747.92732212454
635.5234566946503
484.52063731192806
411.19218961651967
367.29754445749376
303.17713586346287
263.07491107249564
243.50920184749012
223.21093175740629
298.63516907303
220.83102210872667
254.1185442091091
220.42432530395806
236.73473264651594
220.3774163915492
208.70806970275672
207.7465176010051
206.65021110119588
203.78772228625022
201.16082057025764
198.02154450372626
192.3939967893666
184.02868480076927
172.52105867451365
168.10886995717482
164.0942896028302
161.64287872840566
158.28989894853135
157.9542015733456
157.75111598086684
157.7290653674517
157.6844742178201
157.59991875926855
157.4906596304073
157.3809282880323
157.3403138210743
157.31257301117537
157.30989231309096
157.30476517483643
157.29894592881968
157.2862608539989
157.2963579826871
157.28661697919478
157.28695356992955
157.2883649367407
157.28691137976503
157.2862608539989
157.28626120081498
157.2862608539989
157.28626120081498
157.2862608539989
157.28626120081498
157.28626085399

In [163]:
resoptim = solve_ivp(rhs, (0,tend), y0, method='Radau', dense_output=True, jac=rhs_jac, args=(sol.x,))
tplot=np.linspace(0,tend,200)
h1,h2,err=resoptim.sol(tplot)
fig=make_subplots(specs=[[dict(secondary_y=True)]])
fig.add_scatter(x=tplot,y=h1, name='h1')
fig.add_scatter(x=tplot,y=h2, name='h2')
fig.add_scatter(x=ut,y=sol.x,secondary_y=True, name='qcontrol',mode='lines+markers')
fig.update_layout(width=800,height=600,template='plotly_dark')